In [ ]:
from Bio import Entrez, SeqIO
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import pandas as pd
import time
import os

# Configuración de NCBI
Entrez.email = "dreee@gmail.com"

# Virus primario de referencia (ejemplo: PCV1)
REFERENCE_VIRUS_ID = "NC_001792.1"
REFERENCE_SEQUENCE = None

def download_reference_sequence():
    """Descarga la secuencia de referencia"""
    global REFERENCE_SEQUENCE
    try:
        with Entrez.efetch(db="nucleotide", id=REFERENCE_VIRUS_ID, rettype="fasta", retmode="text") as handle:
            REFERENCE_SEQUENCE = SeqIO.read(handle, "fasta")
        print(f"✅ Secuencia de referencia {REFERENCE_VIRUS_ID} descargada")
        return True
    except Exception as e:
        print(f"❌ Error al descargar secuencia de referencia: {str(e)}")
        return False

def calculate_similarity(seq1, seq2):
    """Calcula el porcentaje de similitud entre dos secuencias"""
    alignments = pairwise2.align.globalxx(seq1, seq2, one_alignment_only=True)
    best_alignment = alignments[0]
    matches = best_alignment[2]
    length = max(len(seq1), len(seq2))
    return (matches / length) * 100

def buscar_descargar_circovirus():
    # 1. Descargar secuencia de referencia primero
    if not download_reference_sequence():
        return

    # 2. Búsqueda en NCBI
    query = "(Circoviridae[Organism] OR circovirus[Title]) AND refseq[Filter] AND 1000:3000[SLEN]"
    print(f"🔍 Realizando búsqueda: {query}")
    
    try:
        handle = Entrez.esearch(db="nucleotide", term=query, retmax=10, idtype="acc")
        record = Entrez.read(handle)
        handle.close()
        
        accession_ids = record["IdList"]
        print(f"📊 Se encontraron {len(accession_ids)} secuencias")
        
        if not accession_ids:
            print("❌ No se encontraron secuencias.")
            return

        # 3. Crear directorio para archivos individuales
        os.makedirs("circovirus_sequences", exist_ok=True)
        metadata = []
        
        for acc_id in accession_ids:
            try:
                print(f"⏳ Descargando {acc_id}...")
                with Entrez.efetch(db="nucleotide", id=acc_id, rettype="fasta", retmode="text") as handle:
                    seq_record = SeqIO.read(handle, "fasta")
                
                # Guardar cada secuencia en archivo individual
                individual_file = f"circovirus_sequences/{acc_id}.fasta"
                with open(individual_file, "w") as f:
                    SeqIO.write(seq_record, f, "fasta")
                
                # Calcular similitud con la referencia
                similarity = calculate_similarity(REFERENCE_SEQUENCE.seq, seq_record.seq)
                
                org_name = seq_record.description.split("[")[-1].split("]")[0] if "[" in seq_record.description else "Desconocido"
                metadata.append({
                    "ID": seq_record.id,
                    "Archivo": individual_file,
                    "Descripción": seq_record.description,
                    "Longitud": len(seq_record),
                    "Organismo": org_name,
                    "Similitud_con_Referencia (%)": round(similarity, 2)
                })
                print(f"✅ {seq_record.id} | Similitud: {similarity:.2f}%")
                time.sleep(1)
                
            except Exception as e:
                print(f"❌ Error al procesar {acc_id}: {str(e)}")
                continue

        # 4. Guardar metadatos con resultados de similitud
        if metadata:
            df = pd.DataFrame(metadata)
            df.to_csv("metadata_circovirus.csv", index=False)
            print("\n📊 Resumen de similitudes:")
            print(df[["ID", "Organismo", "Similitud_con_Referencia (%)"]])
            print(f"\n💾 Todos los archivos guardados en 'circovirus_sequences/'")
            print(f"📊 Metadatos guardados en 'metadata_circovirus.csv'")
    
    except Exception as e:
        print(f"❌ Error en la búsqueda/descarga: {str(e)}")

if __name__ == "__main__":
    buscar_descargar_circovirus()